### EDA & Features for SF Crime DATA

In [1]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv("data/train 2.csv")
test = pd.read_csv("data/test.csv")
train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


### Data:

- **Dates**: timestamp of the crime incident
- **Category**: category if the crime incident (only in train.csv) **Response**
- **Descript**: detailed description of the crime incident (only in train.csv)
- **DayofWeek**: the day of the week
- **PdDistrict** : name of the Police Department District
- **Resolution**: how the crime incident was resolved (only in train.csv)
- ** Address**: the approximate street address of the crime incident
- **X**: Longitude
- **Y**: Latitude